In [23]:
# Cell 1: 데이터 로드 및 컬럼 확인
import pandas as pd

df = pd.read_csv('AAA_video_c0.csv')
print("데이터 로드 완료: 행 =", len(df), "열 =", df.shape[1])
print("컬럼 목록:", df.columns.tolist())

데이터 로드 완료: 행 = 6250 열 = 5
컬럼 목록: ['publisher', 'game_names', 'video_type', 'video_link', 'video_url']


In [24]:
# Cell 2: 고유 게임 이름 추출
unique_names = df['game_names'].dropna().unique().tolist()
print("고유 게임 이름 개수:", len(unique_names))

고유 게임 이름 개수: 2445


In [ ]:
# Cell 3: IGDB 인증 준비
import requests
CLIENT_ID = 'ID'       # 실제 Client ID로 교체
CLIENT_SECRET = 'SECRET'  # 실제 Client Secret으로 교체

auth_url = 'https://id.twitch.tv/oauth2/token'
auth_res = requests.post(auth_url, data={
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'grant_type': 'client_credentials'
})
auth_res.raise_for_status()
token = auth_res.json()['access_token']
HEADERS = {
    'Client-ID': CLIENT_ID,
    'Authorization': f'Bearer {token}',
    'Accept': 'application/json'
}
print("IGDB 인증 완료, HEADERS 준비됨")

IGDB 인증 완료, HEADERS 준비됨


In [26]:
# Cell 4 (수정): 출시일 조회 함수 정의 (limit 추가)
def fetch_release_by_names(names):
    """
    names: List[str] - IGDB에서 조회할 게임 이름 리스트
    returns: List[dict] with keys 'name' and 'first_release_date'
    """
    url = 'https://api.igdb.com/v4/games'
    esc = ['"{}"'.format(n.replace('"', '\\"')) for n in names]
    name_list = ",".join(esc)
    # limit을 len(names) 이상으로 설정해서 기본 10개 제한을 해제
    query = (
        f'fields name, first_release_date; '
        f'where name = ({name_list}); '
        f'limit {len(names)};'
    )
    res = requests.post(url, headers=HEADERS, data=query)
    res.raise_for_status()
    return res.json()

In [27]:
# Cell 5: 소규모 테스트 호출 (10개)
test_names = unique_names[:10]
print("테스트 대상 이름 10개:", test_names)
info_test = fetch_release_by_names(test_names)
print("테스트 응답 레코드 수:", len(info_test))
print(info_test)

테스트 대상 이름 10개: ['Resident Evil Outbreak', 'Lost Planet 3', 'Call of Duty: Finest Hour', 'Red Dead Revolver', 'Street Fighter Alpha 3', 'Tatsunoko vs. Capcom: Ultimate All Stars', 'Monster Hunter 3 Ultimate', 'Monster Hunter Tri', 'Street Fighter IV', 'Phoenix Wright: Ace Attorney']
테스트 응답 레코드 수: 10
[{'id': 221285, 'first_release_date': 1369872000, 'name': 'Phoenix Wright: Ace Attorney'}, {'id': 1969, 'first_release_date': 1083456000, 'name': 'Red Dead Revolver'}, {'id': 3761, 'first_release_date': 1100563200, 'name': 'Call of Duty: Finest Hour'}, {'id': 6704, 'first_release_date': 899078400, 'name': 'Street Fighter Alpha 3'}, {'id': 242650, 'first_release_date': 928800000, 'name': 'Street Fighter Alpha 3'}, {'id': 242651, 'first_release_date': 914371200, 'name': 'Street Fighter Alpha 3'}, {'id': 2268, 'first_release_date': 1249084800, 'name': 'Monster Hunter Tri'}, {'id': 1426, 'first_release_date': 1126742400, 'name': 'Phoenix Wright: Ace Attorney'}, {'id': 2271, 'first_release_date':

In [28]:
# Cell 6: 전체 배치 호출 (청크 크기 설정)
import time
all_info = []
CHUNK_SIZE = 50

print("전체 배치 호출 시작...")
for i in range(0, len(unique_names), CHUNK_SIZE):
    chunk = unique_names[i:i+CHUNK_SIZE]
    print(f"  청크 {i}~{i+len(chunk)-1}: 요청 {len(chunk)}개", end=' → ')
    res = fetch_release_by_names(chunk)
    print(f"응답 {len(res)}개")
    all_info.extend(res)
    time.sleep(0.25)
print("전체 수집된 레코드 수:", len(all_info))

전체 배치 호출 시작...
  청크 0~49: 요청 50개 → 응답 50개
  청크 50~99: 요청 50개 → 응답 50개
  청크 100~149: 요청 50개 → 응답 50개
  청크 150~199: 요청 50개 → 응답 50개
  청크 200~249: 요청 50개 → 응답 50개
  청크 250~299: 요청 50개 → 응답 50개
  청크 300~349: 요청 50개 → 응답 50개
  청크 350~399: 요청 50개 → 응답 50개
  청크 400~449: 요청 50개 → 응답 50개
  청크 450~499: 요청 50개 → 응답 50개
  청크 500~549: 요청 50개 → 응답 50개
  청크 550~599: 요청 50개 → 응답 50개
  청크 600~649: 요청 50개 → 응답 50개
  청크 650~699: 요청 50개 → 응답 50개
  청크 700~749: 요청 50개 → 응답 50개
  청크 750~799: 요청 50개 → 응답 50개
  청크 800~849: 요청 50개 → 응답 50개
  청크 850~899: 요청 50개 → 응답 50개
  청크 900~949: 요청 50개 → 응답 50개
  청크 950~999: 요청 50개 → 응답 50개
  청크 1000~1049: 요청 50개 → 응답 50개
  청크 1050~1099: 요청 50개 → 응답 50개
  청크 1100~1149: 요청 50개 → 응답 50개
  청크 1150~1199: 요청 50개 → 응답 50개
  청크 1200~1249: 요청 50개 → 응답 50개
  청크 1250~1299: 요청 50개 → 응답 50개
  청크 1300~1349: 요청 50개 → 응답 50개
  청크 1350~1399: 요청 50개 → 응답 50개
  청크 1400~1449: 요청 50개 → 응답 50개
  청크 1450~1499: 요청 50개 → 응답 50개
  청크 1500~1549: 요청 50개 → 응답 50개
  청크 1550~1599: 요청 50개 → 응답 50개
  청크 1

In [29]:
# Cell 7: full_release_df 생성 및 날짜 변환
full_release_df = pd.DataFrame(all_info)
full_release_df['first_release_date'] = pd.to_datetime(
    full_release_df['first_release_date'], unit='s', errors='coerce'
)
print("full_release_df shape:", full_release_df.shape)

full_release_df shape: (2445, 3)


In [30]:
# Cell 8: 미래·placeholder 날짜 확인 및 개수 출력
today = pd.Timestamp("2025-08-03")
future_mask = full_release_df['first_release_date'] > today
print("미래/placeholder 날짜 건수:", future_mask.sum())

미래/placeholder 날짜 건수: 20


In [31]:
# Cell 9: 이름별 출시일 일관성 확인
unique_release = full_release_df.drop_duplicates(subset=['name','first_release_date'])
counts = unique_release.groupby('name')['first_release_date'].nunique()
inconsistent = counts[counts > 1]
print("여러 날짜 가진 게임 개수:", len(inconsistent))
print("예시:", inconsistent.head().to_dict())

여러 날짜 가진 게임 개수: 235
예시: {'1942': 2, '1943: The Battle of Midway': 2, 'Akumajou Dracula': 3, 'Antarctic Adventure': 2, 'Apollo Justice: Ace Attorney': 2}


In [32]:
# (Cell 10 수정) release_map 재생성
# 1) 미래 날짜 제거
today    = pd.Timestamp("2025-08-03")
valid_df = full_release_df[full_release_df['first_release_date'] <= today]

# 2) 이름별 최소(first) 출시일만 추출 → 유니크 인덱스 확보
release_map = valid_df.groupby('name')['first_release_date'].min()

# 3) 인덱스 유니크 확인
assert release_map.index.is_unique, "아직 중복 인덱스가 있습니다!"

# 4) 원본 df에 매핑 (Cell 11)
df['first_release_date'] = df['game_names'].map(release_map)

In [33]:
# Cell 11
df['first_release_date'] = df['game_names'].map(release_map)

In [34]:
# Cell 12: 매핑 실패(NA) 개수 및 게임 확인
num_missing = df['first_release_date'].isna().sum()
print("매핑 실패(NA) 행 수:", num_missing)
print("NA인 게임 목록 예시:", df.loc[df['first_release_date'].isna(), 'game_names'].unique()[:10])

매핑 실패(NA) 행 수: 920
NA인 게임 목록 예시: ['Resident Evil Outbreak' 'Legend of Kay' 'Viewtiful Joe 2'
 'Mega Man X Collection' 'Harvey Birdman: Attorney at Law' 'Darkwatch'
 'Cadillacs and Dinosaurs' 'Gyakuten Saiban: VR20-gou Jiken'
 '19XX: The War Against Destiny' '1944: The Loop Master']


In [35]:
# Cell 13: 결측치 제거
initial_count = len(df)
df_clean = df.dropna(subset=['first_release_date']).copy()
print(f"결측치 제거 전: {initial_count}행, 제거 후: {len(df_clean)}행")

결측치 제거 전: 6250행, 제거 후: 5330행


In [36]:
df_clean.head(20)

publisher                                game_names            video_type  \
2     Capcom                             Lost Planet 3               Trailer   
3     Capcom                             Lost Planet 3        Gameplay video   
4     Capcom                             Lost Planet 3        Gameplay video   
5     Capcom                 Call of Duty: Finest Hour        Gameplay Video   
6     Capcom                 Call of Duty: Finest Hour               Trailer   
7     Capcom                 Call of Duty: Finest Hour               Trailer   
8     Capcom                 Call of Duty: Finest Hour               Trailer   
9     Capcom                         Red Dead Revolver               Trailer   
10    Capcom                         Red Dead Revolver        Gameplay Video   
11    Capcom                    Street Fighter Alpha 3               Trailer   
12    Capcom                    Street Fighter Alpha 3        Gameplay Video   
13    Capcom                    Street Fighter Alpha 3        Gameplay Video   
14    Capcom  Tatsunoko vs. Capcom: Ultimate All Stars               Trailer   
15    Capcom                 Monster Hunter 3 Ultimate               Trailer   
16    Capcom                 Monster Hunter 3 Ultimate               Trailer   
17    Capcom                 Monster Hunter 3 Ultimate               Trailer   
18    Capcom                        Monster Hunter Tri               Trailer   
19    Capcom                        Monster Hunter Tri               Trailer   
20    Capcom                         Street Fighter IV  Announcement Trailer   
21    Capcom                         Street Fighter IV               Trailer   

     video_link                                    video_url  \
2   igfmUe-IBeQ  https://www.youtube.com/watch?v=igfmUe-IBeQ   
3   8anIGeV18HE  https://www.youtube.com/watch?v=8anIGeV18HE   
4   afZAmzYWEu4  https://www.youtube.com/watch?v=afZAmzYWEu4   
5   2f8S7gPK4b4  https://www.youtube.com/watch?v=2f8S7gPK4b4   
6   qDCADmoCec8  https://www.youtube.com/watch?v=qDCADmoCec8   
7   t7GTci0mT2Q  https://www.youtube.com/watch?v=t7GTci0mT2Q   
8   0JzW38drYic  https://www.youtube.com/watch?v=0JzW38drYic   
9   N-exuahRl2g  https://www.youtube.com/watch?v=N-exuahRl2g   
10  _8yxEqmxOvE  https://www.youtube.com/watch?v=_8yxEqmxOvE   
11  xJwwVeQuZ6I  https://www.youtube.com/watch?v=xJwwVeQuZ6I   
12  mDQSK4aSoWM  https://www.youtube.com/watch?v=mDQSK4aSoWM   
13  hY0lAuhQwzk  https://www.youtube.com/watch?v=hY0lAuhQwzk   
14  qeq-SVAnzNs  https://www.youtube.com/watch?v=qeq-SVAnzNs   
15  z648qIBejT0  https://www.youtube.com/watch?v=z648qIBejT0   
16  7uzafxEHbG0  https://www.youtube.com/watch?v=7uzafxEHbG0   
17  EI75zyCLCG0  https://www.youtube.com/watch?v=EI75zyCLCG0   
18  QE_629ae3iE  https://www.youtube.com/watch?v=QE_629ae3iE   
19  8l3f3csAZn8  https://www.youtube.com/watch?v=8l3f3csAZn8   
20  DJ3ONTRFSqE  https://www.youtube.com/watch?v=DJ3ONTRFSqE   
21  JWhOsX_LAfw  https://www.youtube.com/watch?v=JWhOsX_LAfw   

   first_release_date  
2          2013-08-26  
3          2013-08-26  
4          2013-08-26  
5          2004-11-16  
6          2004-11-16  
7          2004-11-16  
8          2004-11-16  
9          2004-05-02  
10         2004-05-02  
11         1998-06-29  
12         1998-06-29  
13         1998-06-29  
14         2010-01-26  
15         2011-12-10  
16         2011-12-10  
17         2011-12-10  
18         2009-08-01  
19         2009-08-01  
20         2008-07-18  
21         2008-07-18

In [40]:
df_clean.rename(columns={'first_release_date': 'game_release_date'}, inplace=True)

In [42]:
df_clean.head()

publisher                 game_names      video_type   video_link  \
2    Capcom              Lost Planet 3         Trailer  igfmUe-IBeQ   
3    Capcom              Lost Planet 3  Gameplay video  8anIGeV18HE   
4    Capcom              Lost Planet 3  Gameplay video  afZAmzYWEu4   
5    Capcom  Call of Duty: Finest Hour  Gameplay Video  2f8S7gPK4b4   
6    Capcom  Call of Duty: Finest Hour         Trailer  qDCADmoCec8   

                                     video_url game_release_date  
2  https://www.youtube.com/watch?v=igfmUe-IBeQ        2013-08-26  
3  https://www.youtube.com/watch?v=8anIGeV18HE        2013-08-26  
4  https://www.youtube.com/watch?v=afZAmzYWEu4        2013-08-26  
5  https://www.youtube.com/watch?v=2f8S7gPK4b4        2004-11-16  
6  https://www.youtube.com/watch?v=qDCADmoCec8        2004-11-16

In [43]:
df_clean.to_csv('AAA_release.csv', index=False)
print("최종 파일 저장 완료:AAA_release.csv")

최종 파일 저장 완료:AAA_release.csv
